<a href="https://colab.research.google.com/github/ketanhdoshi/ml/blob/master/lib/image_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Image Processing Examples and Utilities

**Todos**
*   Display of Keypt, Bounding Box
*   Augmentation of Keypt, Bounding Box
*   Display of results, including Keypt, Bounding Box
*   Video processing
*   Make image and video utilities independent of torch/keras framework
*   ocv open Gif file code should be general for grey scale
*   ocv byte tensor should handle grey scale 2D and colour 3D. should not permute if 2D
*   ShowImg show_image should handle 3D and 2D greyscale, so mask can be shown as an image
*   Comments for ShowImg
*   Remove WAIT_show_grid and merge show_grid and new_grid















### Import Libraries

In [ ]:
#export

import matplotlib.pyplot as plt
import numpy as np
import math, random

In [ ]:
import IPython.core.debugger as db

from pathlib import Path

import matplotlib.patches as patches
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
# options are 'all', 'none', 'last' and 'last_expr
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
gn_path = 'gdrive/My Drive/Colab Notebooks'  #change dir to your project folder

import sys
sys.path.insert(1, gn_path + '/exp')

### Fetch images for experimentation

In [ ]:
# Download some image files
!wget 'https://live.staticflickr.com/4091/4994221690_d070e8a355_b_d.jpg'
!mv 4994221690_d070e8a355_b_d.jpg mug.jpg

!wget 'https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/01/bondi_beach.jpg'
!mv bondi_beach.jpg beach.jpg

!wget 'http://i.stack.imgur.com/SYxmp.jpg'
!mv SYxmp.jpg girl.jpg

!wget https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/Dryocopus_pileatus_MP2.jpg/600px-Dryocopus_pileatus_MP2.jpg -q -O woodpecker.jpg

from IPython.display import Image, display
display(Image('mug.jpg'))
display(Image('beach.jpg'))

In [ ]:
root_path = Path.cwd()
gd_path = Path.cwd() / 'gdrive' / 'My Drive' / 'Colab Data' / 'fastai-v3' / 'data' / 'augment'

mug_path = root_path/'mug.jpg'
beach_path = root_path/'beach.jpg'
girl_path = root_path/'girl.jpg'
woodpecker_path = root_path / 'woodpecker.jpg'

dog_path = gd_path / 'small dog.png'
dog_mask_path = gd_path / 'small dog mask.png'
parrot_path = gd_path / 'parrot.png'

### Display Utilities

In [ ]:
#export

import torch

class ShowImg():
  # ----------------------------
  # Can display both image tensors and PIL image objects
  # ----------------------------
  @staticmethod
  def show_image(img, ax):
    #if ax is None: _,ax = plt.subplots(1, 1, figsize=figsize)
    ax.axis('off')
    if (isinstance(img, torch.Tensor)):
      img = img.permute(1,2,0)
    ax.imshow(img)

  # ----------------------------
  # ----------------------------
  @staticmethod
  def show_label(label, ax):
    ax.set_title(f'{label}')

  # ----------------------------
  # ----------------------------
  @staticmethod
  def show_mask(mask, ax):
    #!!!! what about OCV mask !!!! mask = mask.convert('RGBA')
    ax.imshow(mask, alpha=0.7, cmap="Reds")

  # ----------------------------
  # 
  # ----------------------------
  @classmethod
  def WAIT_show_grid(cls, x_imgs, y_labels, z_data=None, x_method=None, y_method=None, z_method=None, num_cols=10, figsize=None, **kwargs):
    assert(len(x_imgs) == len(y_labels))
    x_method = cls.show_image if (x_method is None) else x_method
    y_method = cls.show_label if (y_method is None) else y_method
    z_method = cls.show_label if (z_method is None) else z_method

    num_imgs = len(x_imgs)
    num_rows = int (math.ceil (num_imgs / num_cols))
    if (figsize is None):
      figsize=(num_cols * 3, num_rows * 3)
    fig,axes = plt.subplots(num_rows, num_cols, figsize=figsize)

    for img, label, ax in zip (x_imgs, y_labels, axes.flat):
      x_method(img, ax)
      y_method(label, ax)
      #self.show_image(img, ax)
      #self.show_label(label, ax)

    for i in range(len(x_imgs), len(axes.flat)): axes.flat[i].set_visible(False)

  # ----------------------------
  # 
  # ----------------------------
  def show_grid(cls, x_imgs, y_data, z_data=None, x_method=None, y_method=None, z_method=None, num_cols=10, figsize=None, **kwargs):
    x_method = cls.show_image if (x_method is None) else x_method
    y_method = cls.show_label if (y_method is None) else y_method
    z_method = cls.show_label if (z_method is None) else z_method

    if (y_method == cls.show_label):
      yz_type = 'label'
    elif (y_method == cls.show_mask):
      yz_type = 'mask'
    elif (y_method == cls.show_image):
      yz_type = 'image'

    self._new_grid(cls, x_imgs, y_data, z_data, yz_type, num_cols=10, figsize=None, **kwargs)

  # ----------------------------
  # 
  # ----------------------------
  def _new_grid(cls, x_imgs, y_data, z_data=None, yz_type='label', max_num_cols=10, figsize=None, **kwargs):
    assert(len(x_imgs) == len(y_data))
    assert((z_data is None) or (len(x_imgs) == len(z_data)))

    # Number of image sets
    num_img_sets = len(x_imgs)

    # Number of images in each set
    set_len = 1
    if (yz_type == 'image'):
       if (y_data is not None):
         set_len += 1
       if (z_data is not None):
         set_len += 1
    
    elif (yz_type in ['mask', 'bbox', 'keypt']):
      if (z_data is not None):
        set_len += 1

    # Calculate number of rows and columns based on the number of image sets, and
    # the number of images per set.
    num_sets_per_row = max_num_cols // set_len
    num_cols = num_sets_per_row * set_len
    num_rows = int (math.ceil (num_img_sets / num_sets_per_row))

    if (figsize is None):
      figsize=(num_cols * 3, num_rows * 3)
    fig,axes = plt.subplots(num_rows, num_cols, figsize=figsize)

    y_data = y_data if y_data is not None else [None] * num_img_sets
    z_data = z_data if z_data is not None else [None] * num_img_sets

    i = 0
    for x, y, z in zip (x_imgs, y_data, z_data):
      if (yz_type == 'image'):
        i = self._xyz_image(cls, x, y, z, axes.flat, i)
      elif (yz_type == 'label'):
        i = self._xyz_label(cls, x, y, z, axes.flat, i)
      elif (yz_type in ['mask', 'bbox', 'keypt']):
        i = self._xyz_overlay(cls, x, y, z, yz_type, axes.flat, i)
      
      i += 1

    for j in range(i, len(axes.flat)): axes.flat[j].set_visible(False)

  # ----------------------------
  # 
  # ----------------------------
  def _xyz_label(cls, x, y, z, axs_flat, i):
    ax = axs_flat[i] 
    cls.show_image(x, ax)

    label = f'{y}/{z}' if (z is not None) else f'{y}'
    cls.show_label(label, ax)
 
    return i

  # ----------------------------
  # 
  # ----------------------------
  def _xyz_overlay(cls, x, y, z, yz_type, axs_flat, i):
    ax = axs_flat[i] 
    cls.show_image(x, ax)

    if (yz_type == 'mask'):
      yz_method = cls.show_mask
    elif (yz_type == 'bbox'):
      yz_method = cls.show_bbox
    elif (yz_type == 'keypt'):
      yz_method = cls.show_keypt

    if (y is not None):
      yz_method(y, ax)

    if (z is not None):
      i += 1
      ax = axs_flat[i]
      cls.show_image(x, ax)
      yz_method(z > 0, ax)
    
    return i

  # ----------------------------
  # 
  # ----------------------------
  def _xyz_image(cls, x, y, z, axs_flat, i):
    ax = axs_flat[i] 
    cls.show_image(x, ax)

    if (y is not None):
      i += 1
      ax = axs_flat[i]
      cls.show_image(y, ax)

    if (z is not None):
      i += 1
      ax = axs_flat[i]
      cls.show_image(z, ax)

    return i

In [ ]:
#------------------------------------------------------
# Plot an image on an existing figure and axes
#------------------------------------------------------
def plot_image (im, cmap=None):
  if (cmap is None):
    plt.imshow(im)
  else:
    plt.imshow(im, cmap)

#------------------------------------------------------
# Plot two images side-by-side for a before-after comparison
#------------------------------------------------------
def plot_image_compare (im1, im2, cmap1=None, cmap2=None):
  plt.figure(figsize=(10,4))

  plt.subplot(1,2,1)
  plot_image (im1, cmap1)
  plt.axis('off')
  plt.title('Original Image')

  plt.subplot(1,2,2)
  plot_image (im2, cmap2)
  plt.axis('off')
  plt.title('Grayscale Image')

  plt.tight_layout()
  plt.show()
  
#------------------------------------------------------
# Plot the histogram of greyscale values along with a threshold line
#------------------------------------------------------
def plot_hist_threshold (im_grey, thresh):
  im_pixels = im_grey.flatten()
  plt.hist(im_pixels,bins=50)
  plt.vlines(thresh, 0, 100000, linestyle='--')
  plt.ylim([0,50000])
  plt.title('Grayscale Histogram')  
  
#------------------------------------------------------
# Draw a bounding box on an image
#------------------------------------------------------
def bounding_box (ax, x, y, width, height):
  # Create a Rectangle patch
  rect = patches.Rectangle((x,y),width,height,linewidth=1,edgecolor='r',facecolor='none')

  # Add the patch to the Axes
  ax.add_patch(rect)

## Scipy-Image library experiments

In [ ]:
from skimage import io as skio
from skimage.color import rgb2gray
from skimage import exposure
from skimage import filters
from scipy import ndimage

#------------------------------------------------------
# Read an image
#------------------------------------------------------
def read_image (file):
  im = skio.imread(str(file))
  return (im)

#------------------------------------------------------
# Convert an image to greyscale
#------------------------------------------------------
def greyscale (im):
  im_grey = rgb2gray(im)
  return (im_grey)

#------------------------------------------------------
# Segment a greyscale image into foreground and background by calculating
# a separation threshold based on the histogram of pixel intensities
# Also use the threshold to produce a mask of the foreground and background
# Otsu's method, named after Nobuyuki Otsu is used to automatically perform clustering-based 
# image thresholding, or, the reduction of a graylevel image to a binary image. The 
# algorithm assumes that the image contains two classes of pixels following bi-modal 
# histogram (foreground pixels and background pixels), it then calculates the 
# optimum threshold separating the two classes so that their inter-class variance 
# is maximal. Otsu’s method exhibits relatively good performance if the histogram 
# can be assumed to have bimodal distribution and assumed to possess a deep and 
# sharp valley between two peaks
#------------------------------------------------------
def segment_foreback (im_grey):
  thresh = filters.threshold_otsu(im_grey)
  mask = np.where(im_grey > thresh, 1, 0)

  return (mask, thresh)

#------------------------------------------------------
# Get a list of all objects, along with separate masks for each object
# To identify individual objects in the image we separate the connected components
# where all the neighbouring pixels are connected.
# Compute all the separate connected components in the image and label each 
# individual component with a different pixel value ie. a different colour
#------------------------------------------------------
def get_objects_with_mask (im_mask):
  objects, num_objects = ndimage.label(synthetic_mask)
  
  # Make an array of masks, one for each object
  object_masks = []
  for object_idx in range(1, num_objects+1):
    obj_mask = np.where(objects == object_idx, 1, 0)
    object_masks.append(obj_mask)
    
  return (objects, num_objects, object_masks)

### Converting to greyscale

In [ ]:
# Read the image into an array and print the image dimensions
# It has 3 dimensions, where the 3rd dimension is the RGB channel
im = read_image (IMAGE_GIRL)
print('Original image shape: {}, Datatype: {}'.format(im.shape, im.dtype))

# Convert it to a grey scale. It now has only 2 dimensions
im_gray = greyscale (im)
print('New image shape: {}'.format(im_gray.shape))

# Plot the original and greyscale images
plot_image_compare (im, im_gray, None, 'gray')

### Draw a bounding box on the image (with matplotlib, not scipy specific)

In [ ]:
import matplotlib.patches as patches

# Create figure and axes
fig,ax = plt.subplots(1, figsize=(7,5))

# Display the image
ax.imshow(im)

# Draw a bounding box
bounding_box (ax, 280, 10, 230, 320)

plt.show()

### Segmenting into Foreground and Background

In [ ]:
# Segment the image into foreground and background
mask, thresh_val = segment_foreback (im_gray)

plt.figure(figsize=(10,4))

# Plot the mask
plt.subplot(1,2,1)
plt.imshow(mask, cmap='gray', interpolation='nearest')
plt.title('Mask')
plt.axis('off')

# Create a "transparent" mask by blanking out all the white (ie. 0) pixels in the
# initial mask. Superimpose this mask in a different colour on the original image
plt.subplot(1,2,2)
mask_for_display = np.where(mask, mask, np.nan)
plt.imshow(im_gray, cmap='gray')
plt.imshow(mask_for_display, cmap='rainbow', alpha=0.5)
plt.axis('off')
plt.title('Image w/ Transparent Mask')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plot_hist_threshold (im_gray, thresh_val)

plt.subplot(1,2,2)
# Plot the histogram based on the skimage tutorial using skimage.exposure
# Not sure why it looks different from the normal histogram on the left
hist, bins_center = exposure.histogram(im_gray)
plt.plot(bins_center, hist, lw=2)
plt.axvline(thresh_val, color='g', ls='--')

plt.show()

### Identifying individual objects

In [ ]:
# Generate a synthetic image that contains several distinct blobs
def generate_image ():
  np.random.seed(1)
  n = 10
  l = 256
  im = np.zeros((l, l))
  points = l*np.random.random((2, n**2))
  im[(points[0]).astype(np.int), (points[1]).astype(np.int)] = 1
  im = ndimage.gaussian_filter(im, sigma=l/(4.*n))
  return (im)

# Plot bounding boxes for all the objects in the image
def object_boxes (ax, objects):
  for label_idx, (sliceY, sliceX) in enumerate(ndimage.find_objects(objects)):
    x, y = sliceX.start, sliceY.start
    width = sliceX.stop - sliceX.start
    height = sliceY.stop - sliceY.start
    bounding_box (ax, x, y, width, height)

# To identify individual objects in the image, generate a suitable sample image
synthetic_im = generate_image()
# Generate a mask using a threshold of the mean value
synthetic_mask = synthetic_im > synthetic_im.mean()

#labels, nlabels = ndimage.label(synthetic_mask)
labels, nlabels, _ = get_objects_with_mask (synthetic_mask)

print('There are {} separate objects detected.'.format(nlabels))

# Create figure and axes
fig, axes = plt.subplots(1, 4, figsize=(16,5))

# ------- Plot 1 - Original image --------
ax = axes[0]
ax.imshow(synthetic_im)
ax.set_title('Synthetic Image')
ax.axis('off')

# ------- Plot 2 - Mask ------- 
ax = axes[1]
ax.imshow(synthetic_mask, cmap=plt.cm.gray)
ax.set_title('Mask of Mean Pixel Values')
ax.axis('off')

# ------- Plot 3 - Objects based on Connected Components ------- 
ax = axes[2]
# Create a random colormap so that each label value gets a distinct colour
from matplotlib.colors import ListedColormap
rand_cmap = ListedColormap(np.random.rand(256,3))
# Create a transparent mask where non-labeled values are blanked out
labels_for_display = np.where(labels > 0, labels, np.nan)
ax.imshow(synthetic_im, cmap='gray')
ax.imshow(labels_for_display, cmap=rand_cmap)
ax.set_title('Connected Components ({} Objects)'.format(nlabels))
ax.axis('off')

# ------- Plot 4 - Bounding Boxes for all Objects -------
ax = axes[3]
ax.imshow(synthetic_im)
# Draw all the bounding boxes
object_boxes (ax, labels)
ax.set_title('Bounding Boxes ({} Objects)'.format(nlabels))
ax.axis('off')

plt.subplots_adjust(wspace=0.02, hspace=0.02, top=1, bottom=0, left=0, right=1)

plt.show()

In [ ]:
# Display a grid of each object, zoomed in
#-----------------------------------------------------------------------
num_rows, num_cols = 4, 8
fig,axes = plt.subplots(num_rows, num_cols, figsize=(18, 8))

for label_idx, label_coords in enumerate(ndimage.find_objects(labels)):
  # Get the slice of the image corresponding to this object
  cell = synthetic_im[label_coords]

  ax = axes[label_idx // num_cols, label_idx % num_cols]
  ax.imshow(cell)
  ax.set_title ('Object #{}\nSize: {}'.format(label_idx, cell.shape))
  #'Label #{}\nSize: {}'.format(ii+1, cell.shape)
  ax.axis('off')

#plt.subplots_adjust(wspace=0.02, hspace=0.15, top=1, bottom=0, left=0, right=1)
plt.tight_layout()
plt.show()

### Edge Detection

In [ ]:
sobel = filters.sobel(im_gray)
plt.imshow(sobel)

### Gaussian Blurring

In [ ]:
blurred = filters.gaussian(sobel, sigma=2.0)
plt.imshow(blurred)

### This is an example from the scipy tutorials. To extract the largest object. Need to understand it

In [ ]:

np.random.seed(1)
n = 10
l = 256
im = np.zeros((l, l))
points = l*np.random.random((2, n**2))
im[(points[0]).astype(np.int), (points[1]).astype(np.int)] = 1
im = ndimage.gaussian_filter(im, sigma=l/(4.*n))

mask = im > im.mean()

label_im, nb_labels = ndimage.label(mask)

# Find the largest connected component
sizes = ndimage.sum(mask, label_im, range(nb_labels + 1))
mask_size = sizes < 1000
remove_pixel = mask_size[label_im]
label_im[remove_pixel] = 0
labels = np.unique(label_im)
label_im = np.searchsorted(labels, label_im)

# Now that we have only one connected component, extract it's bounding box
slice_x, slice_y = ndimage.find_objects(label_im==4)[0]
roi = im[slice_x, slice_y]

# Original image
plt.figure(figsize=(8, 4))
plt.imshow(roi)
plt.axis('off')

plt.show()

## imageio library experiments

In [ ]:
import imageio

#------------------------------------------------------
# Read an image
#------------------------------------------------------
def read_image (file):
  im = imageio.imread(str(file))
  return (im)

## OpenCV library experiments

In [ ]:
import cv2

#------------------------------------------------------
# Read an image
# The image is read in the BGR colorspace. We have a third dimension as every 
# pixel is represented by it's B, G and R components. This is the default 
# colorspace in which images are read in OpenCV.
#------------------------------------------------------
def read_image (file):
  im = cv2.imread(str(file))
  return (im)

#------------------------------------------------------
# Convert an image to greyscale
# We reduced a dimension when we transformed from the BGR colorspace to grayscale. 
# This is because grayscale is a range of monochromatic shades from black to white. 
# Therefore, a grayscale image contains only shades of gray and no color (i.e it 
# primarily contains only black and white). Transforming the colorspace removes 
# all color information, leaving only the luminance of each pixel. Since digital 
# images are displayed using a combination of red, green, and blue (RGB) colors, 
# each pixel has three separate luminance values. Therefore, these three values 
# must be combined into a single value when removing color from an image. Luminance 
# can also be described as brightness or intensity, which can be measured on a 
# scale from black (zero intensity) to white (full intensity)
#------------------------------------------------------
def greyscale (im):
  im_grey = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
  return (im_grey)

#------------------------------------------------------
# Edge Detection using Sobel Filter
#------------------------------------------------------
def edge_detect_sobel (im_grey):
  #cv2.Sobel arguments - the image, output depth, order of derivative of x, order of derivative of y, kernel/filter matrix size
  sobelx = cv2.Sobel(im_grey,int(cv2.CV_64F),1,0,ksize=3) #ksize=3 means we'll be using the 3x3 Sobel filter
  sobely = cv2.Sobel(im_grey,int(cv2.CV_64F),0,1,ksize=3)
  sobel = np.sqrt(np.square(sobelx) + np.square(sobely))
  return (sobelx, sobely, sobel)

#------------------------------------------------------
# Edge Detection using Canny
#------------------------------------------------------
def edge_detect_canny (im_grey):
  canny = cv2.Canny(im_grey,100, 200)
  return (canny)

In [ ]:
dog_ocv_img = cv2.cvtColor(read_image(dog_path), cv2.COLOR_BGR2RGB)
sc = ShowImg
sc.show_grid([dog_ocv_img], ['open'])

**Resampling Experiments**

In [ ]:
#INTER_NEAREST – a nearest-neighbor interpolation 
#INTER_LINEAR – a bilinear interpolation (used by default) 
#INTER_AREA – resampling using pixel area relation. 
#INTER_CUBIC – a bicubic interpolation over 4×4 pixel neighborhood 
#INTER_LANCZOS4 – a Lanczos interpolation over 8×8 pixel neighborhood

ex_img = dog_ocv_img
resample_types = [cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_AREA, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4]
resampled_labels = ['Original'] + ['NEAREST', 'LINEAR', 'AREA', 'CUBIC', 'LANCZOS4']
resampled_imgs = [ex_img] + [cv2.resize(ex_img, (224,224), interpolation=resample) for resample in resample_types]

sc = ShowImg
sc.show_grid(resampled_imgs, resampled_labels, num_cols=6, figsize=(30, 15))

### Converting to greyscale

In [ ]:
# Read the image into an array and print the image dimensions
# It has 3 dimensions, where the 3rd dimension is the RGB channel
im = read_image (girl_path)
print('Original image shape: {}, Datatype: {}'.format(im.shape, im.dtype))

# Convert it to a grey scale. It now has only 2 dimensions
im_gray = greyscale (im)
print('New image shape: {}'.format(im_gray.shape))

# Plot the original and greyscale images
plot_image_compare (im, im_gray, None, 'gray')

### Edge Detection

In [ ]:
sobelx, sobely, sobel = edge_detect_sobel (im_gray)
canny = edge_detect_canny (im_gray)

# Create figure and axes
fig, axes = plt.subplots(1, 5, figsize=(16,5))

# ------- Plot 1 - Original image --------
ax = axes[0]
ax.imshow(im_gray, cmap='gray')
ax.set_title('Original Image')
ax.axis('off')

# ------- Plot 2 - Sobel X ------- 
ax = axes[1]
ax.imshow(sobelx,cmap='gray')
ax.set_title('Sobel X (vertical edges)')
ax.axis('off')

# ------- Plot 3 - Sobel Y ------- 
ax = axes[2]
ax.imshow(sobely,cmap='gray')
ax.set_title('Sobel Y (horizontal edges)')
ax.axis('off')

# ------- Plot 4 - Combined Sobel Filter -------
ax = axes[3]
ax.imshow(sobel,cmap='gray')
ax.set_title('Sobel Filter')
ax.axis('off')

# ------- Plot 5 - Canny Edge Detection -------
ax = axes[4]
ax.imshow(canny,cmap='gray')
ax.set_title('Canny')
ax.axis('off')

plt.subplots_adjust(wspace=0.02, hspace=0.02, top=1, bottom=0, left=0, right=1)

plt.show()

## Pillow Experiments (Python Image Library, formerly PIL)

In [ ]:
import PIL
from PIL import Image

dog_pil_img = Image.open(dog_path)

**Dihedral Transforms Experiments**

In [ ]:
ex_img = dog_pil_img
transposed_imgs = [ex_img] + [ex_img.transpose(i) for i in range(7)]
transposed_labels = ['Original'] + ['FLIP_LEFT_RIGHT', 'FLIP_TOP_BOTTOM', 'ROTATE_90', 'ROTATE_180', 'ROTATE_270', 'TRANSPOSE', 'TRANSVERSE']

sc = ShowImg
sc.show_grid(transposed_imgs, transposed_labels)

**Resampling Experiments**

In [ ]:
ex_img = dog_pil_img
resample_types = [Image.ANTIALIAS, Image.NEAREST, Image.BICUBIC, Image.BILINEAR]
resampled_labels = ['Original'] + ['ANTIALIAS', 'NEAREST', 'BICUBIC', 'BILINEAR']
resampled_imgs = [ex_img] + [ex_img.resize((224,224), resample=resample) for resample in resample_types]

sc = ShowImg
sc.show_grid(resampled_imgs, resampled_labels, num_cols=5, figsize=(25, 15))

## Image Augmentation (with PIL and OpenCV)

### PIL Augmentation

In [ ]:
#export

import PIL
from PIL import Image
import torch
from torch import tensor

class PilImg():
  #------------------------------------------------------
  # Get (width, height) of the image
  #------------------------------------------------------
  @staticmethod
  def pil_shape(img):
    w, h = img.size
    return w, h

  #------------------------------------------------------
  # Load the file as an image
  #------------------------------------------------------
  @staticmethod
  def pil_open (file):
    img = PIL.Image.open(file)
    return (img)

  #------------------------------------------------------
  # Convert image to RGB format
  #------------------------------------------------------
  @staticmethod
  def pil_rgb (img):
    img_rgb = img.convert('RGB')
    return (img_rgb)

  #------------------------------------------------------
  # Resize image
  # PIL.Image.NEAREST (use nearest neighbour), PIL.Image.BILINEAR (linear interpolation), or PIL.Image.BICUBIC (cubic spline interpolation)
  #------------------------------------------------------
  @staticmethod
  def pil_resize(img, newsz, resample=PIL.Image.BILINEAR):
    assert(isinstance(newsz, tuple) and (len(newsz) == 2))
    assert(resample in [PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC])
    img_rsz = img.resize(newsz, resample=resample) 
    return (img_rsz)

  #------------------------------------------------------
  # List of Flip-Rotate transforms
  #------------------------------------------------------
  pil_fr_types = {
      'rotate_90': PIL.Image.ROTATE_90,
      'rotate_180': PIL.Image.ROTATE_180,
      'rotate_270': PIL.Image.ROTATE_270,
      'fliv_h': PIL.Image.FLIP_LEFT_RIGHT,
      'fliv_v': PIL.Image.FLIP_TOP_BOTTOM,
      'transpose': PIL.Image.TRANSPOSE,
      'transverse': PIL.Image.TRANSVERSE
  }

  #------------------------------------------------------
  # Apply Flip-Rotate transform
  #------------------------------------------------------
  @classmethod
  def pil_flip_rotate(cls, img, fr_type):
    assert (fr_type in ['rotate_90', 'rotate_180', 'rotate_270', 'fliv_h', 'fliv_v', 'transpose', 'transverse'])
    fr_code = cls.pil_fr_types[fr_type]
    fr_img = img.transpose(fr_code)

    return (fr_img)

  #------------------------------------------------------
  # Crop image
  #------------------------------------------------------
  @staticmethod
  def pil_crop(img, newsz, crop_corners, resample=PIL.Image.BILINEAR):
    assert(isinstance(newsz, tuple) and (len(newsz) == 2))
    assert(isinstance(crop_corners, tuple) and (len(crop_corners) == 4))
    assert(resample in [PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC])
    img_crop = img.transform(newsz, PIL.Image.EXTENT, crop_corners, resample=resample)
    return (img_crop)

  #------------------------------------------------------
  # Perspective transform
  #------------------------------------------------------
  @staticmethod
  def pil_perspective(img, newsz, matrix, resample=PIL.Image.BILINEAR):
    assert(isinstance(newsz, tuple) and (len(newsz) == 2))
    pers_img = img.transform(newsz, PIL.Image.PERSPECTIVE, list(matrix), resample=resample)

    return (pers_img)

  #------------------------------------------------------
  # Convert image to a tensor of bytes
  #------------------------------------------------------
  @staticmethod
  def pil_byte_tensor(img):
    # PIL .tobytes() converts the image object to raw data, one byte per pixel
    res = torch.ByteTensor(torch.ByteStorage.from_buffer(img.tobytes()))
    w,h = img.size
    return res.view(h,w,-1).permute(2,0,1)

In [ ]:
pil_cls = PilImg
open_img = pil_cls.pil_open(dog_path)
rgb_img = pil_cls.pil_rgb(open_img)
rsz_img = pil_cls.pil_resize(rgb_img, (128, 128))
fr_img = pil_cls.pil_flip_rotate(rsz_img, 'rotate_180')

sc = ShowImg
sc.show_grid([open_img, rgb_img, rsz_img, fr_img], ['open', 'rgb', 'resize', 'flip_rotate'])
pil_cls.pil_shape(open_img), pil_cls.pil_shape(rgb_img), pil_cls.pil_shape(rsz_img)

### OpenCV Augmentation

In [ ]:
#export

import cv2
import imageio
import torch
from torch import tensor

class OcvImg():
  #------------------------------------------------------
  # Get (width, height) of the image
  #------------------------------------------------------
  @staticmethod
  def ocv_shape(img):
    h, w, d = img.shape
    return w, h

  #------------------------------------------------------
  # Load the file as an image
  #------------------------------------------------------
  @staticmethod
  def ocv_open (file):
    if (str(file)[-4:] == '.gif'):
      # OpenCV doesn't support .gif codec. So read the file into a numpy array 
      # with ImageIO and convert from RGB to BGR, so that it is in OpenCV format
      img = imageio.imread(file)
      img = img.astype('float32') 
      #img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
      return (img)

    img = cv2.imread(str(file))
    return (img)

  #------------------------------------------------------
  # Convert image to RGB format
  #------------------------------------------------------
  @staticmethod
  def ocv_rgb (img):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (img_rgb)

  #------------------------------------------------------
  # Resize image
  # To shrink an image, it will generally look best with INTER_AREA interpolation, 
  # whereas to enlarge an image, it will generally look best with INTER_CUBIC (slow) 
  # or INTER_LINEAR (faster but still looks OK)
  #------------------------------------------------------
  @staticmethod
  def ocv_resize(img, new_sz, interpolation=cv2.INTER_LINEAR):
    assert(isinstance(new_sz, tuple) and (len(new_sz) == 2))
    img_rsz = cv2.resize(img, new_sz, interpolation=interpolation) 
    return (img_rsz)

  #------------------------------------------------------
  # List of Flip-Rotate transform
  #------------------------------------------------------
  ocv_fr_types = {
      'rotate_90': [(cv2.rotate, cv2.ROTATE_90_CLOCKWISE)],
      'rotate_180': [(cv2.rotate, cv2.ROTATE_180)],
      'rotate_270': [(cv2.rotate, cv2.ROTATE_90_COUNTERCLOCKWISE)],
      'fliv_h': [(cv2.flip, 1)],
      'fliv_v': [(cv2.flip, 0)],
      'transpose': [(cv2.rotate, cv2.ROTATE_90_CLOCKWISE), (cv2.flip, 1)],
      'transverse': [(cv2.rotate, cv2.ROTATE_90_COUNTERCLOCKWISE), (cv2.flip, 1)]
  }

  #------------------------------------------------------
  # Apply Flip-Rotate transform
  #------------------------------------------------------
  @classmethod
  def ocv_flip_rotate(cls, img, fr_type):
    assert (fr_type in ['rotate_90', 'rotate_180', 'rotate_270', 'fliv_h', 'fliv_v', 'transpose', 'transverse'])
    for fr_fn, fr_code in cls.ocv_fr_types[fr_type]:
      img = fr_fn(img, fr_code)

    return (img)

  #------------------------------------------------------
  # Crop image
  #------------------------------------------------------
  @staticmethod
  def ocv_crop(img, new_sz, crop_corners, interpolation=cv2.INTER_LINEAR):
    assert(isinstance(new_sz, tuple) and (len(new_sz) == 2))
    assert(isinstance(crop_corners, tuple) and (len(crop_corners) == 4))
    assert(interpolation in [cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_AREA, cv2.INTER_CUBIC, cv2.INTER_LANCZOS4])
    tl_x, tl_y, br_x, br_y = crop_corners
    img_crop = img[tl_y:br_y, tl_x:br_x]
    img_crop = cv2.resize(img_crop, new_sz, interpolation=interpolation)
    return (img_crop)

  #------------------------------------------------------
  # Perspective transform
  #------------------------------------------------------
  @staticmethod
  def ocv_perspective(img, new_sz, src_coords, targ_coords, interpolation=cv2.INTER_LINEAR):
    assert(isinstance(new_sz, tuple) and (len(new_sz) == 2))
    matrix = cv2.getPerspectiveTransform(np.float32(src_coords), np.float32(targ_coords))
    # Switch source and target for OpenCV to get the same matrix coefficients as for PIL
    #cv2d_flip = cv2.getPerspectiveTransform(np.float32(targ_d), np.float32(src_d))
    pers_img = cv2.warpPerspective(img, matrix, new_sz)

    return (pers_img)

  #------------------------------------------------------
  # Convert image to a tensor of bytes
  #------------------------------------------------------
  @staticmethod
  def ocv_byte_tensor(img):
    img_bytes = torch.from_numpy(img.transpose((2, 0, 1)))
    return (img_bytes)

In [ ]:
ocv_cls = OcvImg
open_img = ocv_cls.ocv_open(dog_path)
rgb_img = ocv_cls.ocv_rgb(open_img)
rsz_img = ocv_cls.ocv_resize(rgb_img, (128, 128))
fr_img = ocv_cls.ocv_flip_rotate(rsz_img, 'rotate_180')

sc = ShowImg
sc.show_grid([open_img, rgb_img, rsz_img, fr_img], ['open', 'rgb', 'resize', 'flip_rotate'])
ocv_cls.ocv_shape(open_img), ocv_cls.ocv_shape(rgb_img), ocv_cls.ocv_shape(rsz_img)

**Dihedral Transforms Experiments**

In [ ]:
ex_img = rgb_img
ocv_cls = OcvImg
transposed_imgs = [ex_img] + [ocv_cls.ocv_flip_rotate(ex_img, fr_type) for fr_type in ocv_cls.ocv_fr_types.keys()]
transposed_labels = ['Original'] + [fr_type for fr_type in ocv_cls.ocv_fr_types.keys()]

sc = ShowImg
sc.show_grid(transposed_imgs, transposed_labels)

### Crop and Warp Calculations

**Resized Crop**

In [ ]:
#export

# Take a crop from the original image and then resize that crop to a given 'final_sz', as 
# all final images should be the same size
def imgaug_random_resized_crop(orig_sz, crop_area_range=(0.08, 1.0), crop_aspect_range=(3./4., 4./3.), crop_pos='random'):

  # We are given three parameters to calculate the position and size of the cropped image
  #   1. The area of the crop as a percentage of the original image
  #         We are given a range of percentages and pick one at random
  #   2. The aspect ratio of the crop
  #         We are given a range of aspect ratios and pick one at random
  #         If no range is given, we take the aspect ratio of the original image
  #   3. The position of the crop within the original image
  #         We either centre the crop within the original image, or find a position at
  #         random
  assert(isinstance(crop_area_range, tuple) and (len(crop_area_range) == 2))
  assert((crop_area_range[0] > 0) and (crop_area_range[0] <= crop_area_range[1]) and (crop_area_range[1] <= 1.))
  assert(crop_aspect_range is None or 
         (isinstance(crop_aspect_range, tuple) and (len(crop_aspect_range) == 2)))
  assert(crop_aspect_range is None or 
         ((crop_aspect_range[0] > 0.5) and (crop_aspect_range[0] < crop_aspect_range[1]) and (crop_aspect_range[1] < 1.5)))
  assert(crop_pos in ['random', 'ctr'])

  orig_w, orig_h = orig_sz
  orig_area = orig_w * orig_h
  min_crop_area_pct, max_crop_area_pct = crop_area_range
  if (crop_aspect_range is not None):
    min_crop_aspect, max_crop_aspect = crop_aspect_range
  else:
    orig_aspect_ratio = orig_w / orig_h

  # Try 'num_attempts' times to get a proper crop inside the image
  num_attempts = 10
  for i in range(num_attempts):

    # Get the crop area as a random percentage of the original image area
    crop_area = random.uniform(min_crop_area_pct, max_crop_area_pct) * orig_area

    # Get the crop aspect ratio
    if (crop_aspect_range is not None):
      # Get a random crop aspect ratio within the given range
      crop_aspect = math.exp(random.uniform(math.log(min_crop_aspect), math.log(max_crop_aspect)))
    else:
      # No range has been provided, so the crop aspect ratio will be the same as the original image
      crop_aspect = orig_aspect_ratio

    # Calculate crop width and height from the crop area and aspect ratio
    #   crop_aspect = crop_w / crop_h
    #   crop_area = crop_w * crop_h
    crop_w = int(round(math.sqrt(crop_area * crop_aspect)))
    crop_h = int(round(math.sqrt(crop_area / crop_aspect)))

    # If the crop dimensions fit within the original image, get the crop position
    if (crop_w <= orig_w and crop_h <= orig_h):

      if (crop_pos == 'random'):
        # Get a random (left, top) position
        crop_pos_l = random.randint(0, orig_w - crop_w)
        crop_pos_t  = random.randint(0, orig_h - crop_h)

      elif (crop_pos == 'ctr'):
        # Get the (left, top) position if the crop is centred
        crop_pos_l = (orig_w - crop_w) // 2
        crop_pos_t = (orig_h - crop_h) // 2

      # Crop (right, bottom) position
      crop_pos_r = crop_pos_l + crop_w
      crop_pos_b = crop_pos_t + crop_h

      # Return coordinates
      return (crop_pos_l, crop_pos_t, crop_pos_r, crop_pos_b)

  # We weren't able to get an appropriate crop after all our attempts
  # So we fallback to squishing
  # 
  # We will only get here if a crop aspect range was given and all the
  # random crop aspect ratios that we attempted did not work
  assert(crop_aspect_range is not None)
  if orig_aspect_ratio < min_crop_aspect:
    # Original aspect ratio was lower than minimum ie. image is narrow and tall
    crop_aspect = min_crop_aspect
    crop_w = orig_w
    crop_h = int(crop_w / crop_aspect)

  elif orig_aspect_ratio > max_crop_aspect:
    # Original aspect ratio was higher than maximum ie. image was wide and short
    crop_aspect = max_crop_aspect
    crop_h = orig_h
    crop_w = int(crop_h * crop_aspect)

  else:
    crop_aspect = orig_aspect_ratio
    crop_w, crop_h = orig_w, orig_h

  crop_pos_l = (orig_w - crop_w) // 2
  crop_pos_t = (orig_h - crop_h) // 2
  crop_pos_r = crop_pos_l + crop_w
  crop_pos_b = crop_pos_t + crop_h

  return (crop_pos_l, crop_pos_t, crop_pos_r, crop_pos_b)

In [ ]:
# Does a centre crop of approx 77% of the image area
ex_img = dog_pil_img
ex_sz = ex_img.size
imgaug_random_resized_crop(ex_sz, crop_area_range=(0.7694, 0.7694), crop_aspect_range=None, crop_pos='ctr')

random.seed(800)
imgaug_random_resized_crop(ex_sz)

**Perspective Warp**

In [ ]:
#export
from torch import FloatTensor

def _find_coeffs(orig_pts, targ_pts):
    matrix = []
    #The equations we'll need to solve.
    for p1, p2 in zip(targ_pts, orig_pts):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = FloatTensor(matrix)
    B = FloatTensor(orig_pts).view(8, 1)
    #The 8 scalars we seek are solution of AX = B
    return list(torch.solve(B,A)[0][:,0])

def _warp(final_sz, src_coords):
    w,h = final_sz
    targ_coords = ((0,0),(0,h),(w,h),(w,0))
    coeffs = _find_coeffs(src_coords,targ_coords)
    return coeffs, src_coords, targ_coords

def _uniform(a,b): return a + (b-a) * random.random()

def _default_crop_size(w,h): return [w,w] if w < h else [h,h]

def imgaug_perspective_warp(orig_sz, final_sz, magnitude=0.):
  orig_w, orig_h = orig_sz
  crop_w, crop_h = _default_crop_size(orig_w, orig_h)

  left, top = random.randint(0, orig_w - crop_w), random.randint(0, orig_h - crop_h)
  top_magn = min(magnitude, left / crop_w, (orig_w - left) / crop_w - 1)
  lr_magn  = min(magnitude, top / crop_h, (orig_h - top) / crop_h - 1)

  up_t, lr_t = _uniform(-top_magn, top_magn), _uniform(-lr_magn, lr_magn)
  src_corners = tensor([[-up_t, -lr_t], [up_t, 1+lr_t], [1-up_t, 1-lr_t], [1+up_t, lr_t]])
  src_corners = src_corners * tensor([crop_w, crop_h]).float() + tensor([left, top]).float()
  src_corners = tuple([(int(o[0].item()), int(o[1].item())) for o in src_corners])
  
  res = _warp(final_sz, src_corners)
  return res

In [ ]:
ex_img = dog_pil_img
ex_sz = ex_img.size
imgaug_perspective_warp(ex_sz, (128, 128), magnitude=0.2)

### PIL and OpenCV Crop and Warp Experiments

In [ ]:
#------------------------------------------------------
# Load an PIL and OCV image, convert them to byte tensors
# and check that the tensors are identical
#------------------------------------------------------
def load_pil_ocv_img(img_path):
  pil_cls = PilImg
  pil_img = pil_cls.pil_open(img_path)
  pil_img = pil_cls.pil_rgb(pil_img)
  pil_tensor = pil_cls.pil_byte_tensor(pil_img)

  ocv_cls = OcvImg
  ocv_img = ocv_cls.ocv_open(img_path)
  ocv_img = ocv_cls.ocv_rgb(ocv_img)
  ocv_tensor = ocv_cls.ocv_byte_tensor(ocv_img)

  return pil_img, ocv_img, pil_tensor, ocv_tensor

pil_img, ocv_img, pil_tensor, ocv_tensor = load_pil_ocv_img(dog_path)

pil_tensor.shape, ocv_tensor.shape
print ('\n Equality of PIL and OCV tensor:', torch.all(torch.eq(pil_tensor, ocv_tensor)))

**Crop and Warp**

In [ ]:
def test_crop(img, img_sz, img_type='ocv'):
  assert(img_type in ['ocv', 'pil'])

  cropped_imgs, cropped_corners = [], []
  new_sz = (128, 128)
  base_crop_area=(0.9, 0.9)

  num_count = 20
  for i in range(num_count):
    if (i < num_count // 2):
      # Centre crop
      crop_scale = base_crop_area[0] - (i * 0.08)
      crop_area=(crop_scale, crop_scale)
      crop_corners = imgaug_random_resized_crop(img_sz, crop_area_range=crop_area, crop_aspect_range=None, crop_pos='ctr')
      cropped_corners.append(f'Ctr, {crop_scale:.2f}, {crop_corners}')
    else:
      # Random resize crop
      crop_corners = imgaug_random_resized_crop(img_sz)
      cropped_corners.append(f'Rnd, {crop_corners}')

    if (img_type == 'ocv'):
      img_crop = OcvImg.ocv_crop(img, new_sz, crop_corners, interpolation=cv2.INTER_LINEAR)
    elif (img_type == 'pil'):
      img_crop = PilImg.pil_crop(img, new_sz, crop_corners, resample=PIL.Image.BILINEAR)
    cropped_imgs.append(img_crop)

  sc = ShowImg
  sc.show_grid(cropped_imgs, cropped_corners)

def test_warp(img, img_sz, img_type='ocv'):
  assert(img_type in ['ocv', 'pil'])

  warped_imgs, warped_pts = [], []
  new_sz = (128, 128)

  for i in range(8):
    coeffs, src_coords, targ_coords = imgaug_perspective_warp(img_sz, new_sz, magnitude=0.2)
    if (img_type == 'ocv'):
      img_warp = OcvImg.ocv_perspective(img, new_sz, src_coords, targ_coords, interpolation=cv2.INTER_LINEAR)
    elif (img_type == 'pil'):
      img_warp = PilImg.pil_perspective(img, new_sz, list(coeffs), resample=PIL.Image.BILINEAR)
    warped_imgs.append(img_warp)
    warped_pts.append(f'W, {src_coords}')

  sc = ShowImg
  sc.show_grid(warped_imgs, warped_pts)

**Crop with PIL**

In [ ]:
ex_img = pil_img
ex_sz = ex_img.size
test_crop(ex_img, ex_sz, img_type='pil')

**Warp with PIL**

In [ ]:
ex_img = pil_img
ex_sz = ex_img.size
test_warp(ex_img, ex_sz, img_type='pil')

**Crop with OCV**

In [ ]:
ex_img = ocv_img
ex_sz = OcvImg.ocv_shape(ex_img)
test_crop(ex_img, ex_sz, img_type='ocv')

**Warp with OCV**

In [ ]:
ex_img = ocv_img
ex_sz = OcvImg.ocv_shape(ex_img)
test_warp(ex_img, ex_sz, img_type='ocv')

### Image Augmentation with Albumentations

In [ ]:
import imageio

def load_example_images(dog_path, dog_mask_path, parrot_path, woodpecker_path):

  # load mask for sample dog image
  dog_pil_img = Image.open(dog_path)
  dog_pil_mask = Image.open(dog_mask_path)

  dog_iio_img = imageio.imread(dog_path)
  dog_iio_mask = imageio.imread(dog_mask_path)

  # Convert Path object to str, as imread() requires
  parrot_ocv_img = cv2.imread(str(parrot_path))
  parrot_ocv_img = cv2.cvtColor(parrot_ocv_img, cv2.COLOR_BGR2RGB)
  parrot_ocv_img = cv2.resize(parrot_ocv_img, (512, 512))

  woodpecker_ocv_img = cv2.imread(str(woodpecker_path))
  woodpecker_ocv_img = cv2.cvtColor(woodpecker_ocv_img, cv2.COLOR_BGR2RGB)
  woodpecker_pcv_img = cv2.resize(woodpecker_ocv_img, (512, 512))

  return dog_pil_img, dog_pil_mask, dog_iio_img, dog_iio_mask, parrot_ocv_img, woodpecker_ocv_img


dog_pil_img, dog_pil_mask, dog_iio_img, dog_iio_mask, parrot_ocv_img, woodpecker_ocv_img = load_example_images(dog_path, dog_mask_path, parrot_path, woodpecker_path)
sc = ShowImg
sc.show_grid([dog_pil_img, dog_pil_mask, dog_iio_img, dog_iio_mask, parrot_ocv_img, woodpecker_ocv_img], 
             ['PIL ' + dog_path.name, 'PIL ' + dog_mask_path.name, 'IIO ' + dog_path.name, 'IIO ' + dog_mask_path.name, 'OCV ' + parrot_path.name, 'OCV ' + woodpecker_path.name])

In [ ]:
import matplotlib.patches as patches
import albumentations as A

def show_dog_with_mask (dog_img, dog_mask):
  fig, axs = plt.subplots(1, 4, figsize=(20,5))

  #plot the original data
  axs[0].imshow(dog_img) 
  axs[0].axis('off')
  axs[0].set_title('Image')

  #plot the mask
  axs[1].imshow(dog_mask)
  axs[1].axis('off')   
  axs[1].set_title('Mask')
    
  #plot image and add the mask
  axs[2].imshow(dog_img)
  axs[2].imshow(dog_mask, alpha = 0.5, cmap = "Reds")
  axs[2].axis('off')   
  axs[2].set_title('Image with mask overlay')

  # Display the image
  axs[3].imshow(dog_img)
  # Create a Rectangle patch
  rect = patches.Rectangle((0,1),236,128,linewidth=1,edgecolor='r',facecolor='none')
  # Add the patch to the Axes
  axs[3].add_patch(rect)
  axs[3].axis('off') # disable axis

  # set suptitle
  plt.suptitle('Image with mask')
  plt.show()

show_dog_with_mask (dog_pil_img, dog_pil_mask)

In [ ]:
def gallery(array, ncols=3):
    '''
    Function to arange images into a grid.
    INPUT:
        array - numpy array containing images
        ncols - number of columns in resulting imahe grid
    OUTPUT:
        result - reshaped array into a grid with given number of columns
    '''
    nindex, height, width, intensity = array.shape
    nrows = nindex//ncols
    assert nindex == nrows*ncols
    result = (array.reshape(nrows, ncols, height, width, intensity)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols, intensity))
    return result

def albu_aug (image):
  # initialize augmentations
  gaus_noise = A.GaussNoise() # gaussian noise
  elastic = A.ElasticTransform() # elastic transform
  bright_contrast = A.RandomBrightnessContrast(p=1) # random brightness and contrast
  gamma = A.RandomGamma(p=1) # random gamma
  clahe = A.CLAHE(p=1) # CLAHE (see https://en.wikipedia.org/wiki/Adaptive_histogram_equalization#Contrast_Limited_AHE)
  blur = A.Blur()

  # apply augmentations
  # pass image to the augmentation
  img_gaus = gaus_noise(image = image)
  img_elastic = elastic(image = image)
  img_bc = bright_contrast(image = image)
  img_gamma = gamma(image = image)
  img_clahe = clahe(image = image)
  img_blur = blur(image = image)

  # access the augmented image by 'image' key
  img_list = [img_gaus['image'], img_elastic['image'], img_bc['image'], img_gamma['image'], img_clahe['image'], img_blur['image']]

  # visualize the augmented images
  plt.figure(figsize=(10,10))
  plt.axis('off')
  plt.imshow(gallery(np.array(img_list), ncols = 3))
  plt.title('Augmentation examples')

albu_aug(dog_iio_img)

In [ ]:
def albu_aug_with_mask (image, mask):
  # compose augmentation pipeline
  aug_pipeline = A.Compose([
      A.ShiftScaleRotate(p = 1),
      A.RGBShift(),
      A.Blur(p = 1),
      A.GaussNoise(p = 1)
  ],p=1)

  # apply augmentations to image and a mask
  augmented = aug_pipeline(image = image, mask = mask)

  # visualize augmented image and mask
  fig, ax = plt.subplots(1,4, figsize = (15, 10))

  ax[0].axis('off')
  ax[0].imshow(image)
  ax[0].set_title('original image')

  ax[1].axis('off')
  ax[1].imshow(augmented['image'])
  ax[1].set_title('augmented image')

  ax[2].axis('off')
  ax[2].imshow(augmented['image'])
  ax[2].imshow(augmented['mask'].squeeze(), alpha = 0.5, cmap = "Reds")
  ax[2].set_title('augmented image with mask')

  ax[3].axis('off')
  ax[3].imshow(augmented['mask'].squeeze(), alpha = 1.0, cmap = "Reds")
  ax[3].set_title('augmented mask')

albu_aug_with_mask (dog_iio_img, dog_iio_mask)

In [ ]:
# create bounding boxes from mask with cv2
import cv2

def albu_aug_with_bbox(image, mask):
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
  bboxes = cv2.boundingRect(cv2.findNonZero(mask))

  # compose augmentation pipeline
  aug_pipeline = A.Compose([
      A.ShiftScaleRotate(rotate_limit=0, p = 1),
      A.RGBShift(p = 1),
      A.Blur(p = 1),
      A.GaussNoise(p = 1)
  ],p=1)

  # augment image and bounding box
  augmented_boxes = aug_pipeline(image = image, bboxes = [bboxes])
  box_aug = augmented_boxes['bboxes'][0]

  # visualize augmented image and bbox
  fig, ax = plt.subplots(1,2, figsize = (15, 10))

  ax[0].axis('off')
  ax[0].imshow(image)
  rect = patches.Rectangle((bboxes[0],bboxes[1]),bboxes[2],bboxes[3],linewidth=1,edgecolor='r',facecolor='none')
  ax[0].add_patch(rect)
  ax[0].set_title('original image')

  ax[1].axis('off')
  ax[1].imshow(augmented_boxes['image'])
  rect = patches.Rectangle((box_aug[0],box_aug[1]),box_aug[2],box_aug[3],linewidth=1,edgecolor='r',facecolor='none')
  ax[1].add_patch(rect)
  ax[1].set_title('augmented image')

albu_aug_with_bbox(dog_iio_img, dog_iio_mask)

In [ ]:
def albu_aug_pipeline(image):
  # compose complex augmentation pipeline
  augmentation_pipeline = A.Compose(
      [
          A.HorizontalFlip(p = 0.5), # apply horizontal flip to 50% of images
          A.OneOf(
              [
                  # apply one of transforms to 50% of images
                  A.RandomContrast(), # apply random contrast
                  A.RandomGamma(), # apply random gamma
                  A.RandomBrightness(), # apply random brightness
              ],
              p = 0.5
          ),
          A.OneOf(
              [
                  # apply one of transforms to 50% images
                  A.ElasticTransform(
                      alpha = 120,
                      sigma = 120 * 0.05,
                      alpha_affine = 120 * 0.03
                  ),
                  A.GridDistortion(),
                  A.OpticalDistortion(
                      distort_limit = 2,
                      shift_limit = 0.5
                  ),
              ],
              p = 0.5
          )
      ],
      p = 1
  )

  # apply pipeline to sample image
  images_aug = np.array([augmentation_pipeline(image = image)['image'] for _ in range(16)])

  # visualize augmentation results
  plt.figure(figsize=(10,10))
  plt.axis('off')
  plt.imshow(gallery(images_aug, ncols = 4))
  plt.title('Augmentation pipeline examples')

albu_aug_pipeline(dog_iio_img)

In [ ]:
#export
import albumentations as A

albu_augs = {'Vertical Flip': A.VerticalFlip(p=0.5),
             'Horizontal Flip': A.HorizontalFlip(p=0.5),
             'Flip': A.Flip(p=0.5),
             'Random Rotate': A.RandomRotate90(p=0.5),
             'Rotate': A.Rotate(limit=286, p=0.5),
             'Transpose': A.Transpose(p=0.5),
             'Shift Scale Rotate': A.ShiftScaleRotate(shift_limit=0.8, scale_limit=1.4, rotate_limit=360, p=0.5),
             'Center Crop': A.CenterCrop(height=134, width=94, p=0.5),
             'Random Brightness': A.RandomBrightness(limit=1.3, p=0.5),
             'Random Brightness Contrast': A.RandomBrightnessContrast(p=0.5),
             'Random Gamma': A.RandomGamma(p=0.5),
             'Clahe': A.CLAHE(p=0.5),
             'Hue Saturation Value': A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=0.5),
             'RGB Shift': A.RGBShift(r_shift_limit=105, g_shift_limit=45, b_shift_limit=40, p=0.5),
             'Channel Shuffle': A.ChannelShuffle(p=0.5),
             'Jpeg Compression': A.JpegCompression(quality_lower=7, quality_upper=100, p=0.5),
             'Random Contrast': A.RandomContrast(limit=0.9, p=0.5),
             'Blur': A.Blur(blur_limit=17, p=0.5),
             'Gauss Noise': A.GaussNoise(var_limit=(10.0, 80.0), p=0.5),
             'Invert Image': A.InvertImg(),
}

def WAIT_imgaug_albu(img, aug_name, p=0.5, **kwargs):
  aug = albu_augs[aug_name]
  aug.p = p
  augmented = aug(image=img)
  return augmented['image']

def imgaug_albu(img, aug_name, mask=None, p=0.5, **kwargs):
  aug = albu_augs[aug_name]
  aug.p = p
  augmented = aug(image=img, mask=mask)
  return augmented['image'], augmented['mask']

In [ ]:
aug_names = list(albu_augs.keys())
aug_labels = ['Original'] + aug_names
aug_imgs = [woodpecker_ocv_img] + [imgaug_albu(woodpecker_ocv_img, aug_name, p=1.0)[0] for aug_name in aug_names]

sc = ShowImg
sc.show_grid(aug_imgs, aug_labels)

In [ ]:
aug_names = list(albu_augs.keys())
aug_labels = ['Original'] + aug_names
aug_imgs = [(dog_iio_img, dog_iio_mask)] + [imgaug_albu_mask(dog_iio_img, dog_iio_mask, aug_name, p=1.0) for aug_name in aug_names]

foo_imgs, foo_masks = zip(*aug_imgs)
sc = ShowImg
sc.show_grid(foo_imgs, foo_masks, y_method=sc.show_mask)
sc.show_grid(foo_imgs, aug_labels)

### (TODO - convert to OpenCV) Segmenting into Foreground and Background 

In [ ]:
# Segment the image into foreground and background
# TODO - Replace with OpenCV equivalent
# mask, thresh_val = segment_foreback (im_gray)

plt.figure(figsize=(10,4))

# Plot the mask
plt.subplot(1,2,1)
plt.imshow(mask, cmap='gray', interpolation='nearest')
plt.title('Mask')
plt.axis('off')

# Create a "transparent" mask by blanking out all the white (ie. 0) pixels in the
# initial mask. Superimpose this mask in a different colour on the original image
plt.subplot(1,2,2)
mask_for_display = np.where(mask, mask, np.nan)
plt.imshow(im_gray, cmap='gray')
plt.imshow(mask_for_display, cmap='rainbow', alpha=0.5)
plt.axis('off')
plt.title('Image w/ Transparent Mask')

plt.tight_layout()
plt.show()

In [ ]:
#Distribution of the intensity values of all the pixels
# TODO - make this into a function
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
sns.distplot(im_gray.flatten(),kde=False)#This is to flatten the matrix and put the intensity values of all the pixels in one single row vector
plt.title('Distribution of intensity values')

#To zoom in on the distribution and see if there is more than one prominent peak 
plt.subplot(1,2,2)
sns.distplot(im_gray.flatten(),kde=False) 
plt.ylim(0,30000) 
plt.title('Distribution of intensity values (Zoomed In)')

### Export

In [ ]:
!wget https://raw.githubusercontent.com/ketanhdoshi/ml/master/lib/nb_export.py

In [ ]:
from nb_export import notebook2scriptSingle
notebook2scriptSingle(gn_path + '/lib/image_lib.ipynb', gn_path + '/exp')